In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.engine.sequential import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD

In [5]:
data_path = 'data/2017-20211231_30min.csv'
data = pd.read_csv(data_path)
data = data[(data['timestamp'] >= '2021-01-01') & (data['timestamp'] <= '2021-12-31')]
data = data.drop(['timestamp'], axis=1)

data.head()

,price_open,price_high,price_low,price_close,volume_traded,MACD,SMA,OBV,RSI,MFI,vader_pos,vader_neg,vader_neu,vader_compound,pos_count,neg_count,neu_count
58464,28999.63,29086.90,28774.64,28910.87,146.380654,1949.158199,29042.940,-54448.14249,52.423647,92.220337,0.0,0.0,1.0,0.0,0.0,0.0,0.0
58465,28900.11,29099.00,28900.11,29056.94,251.584121,1960.810507,29033.594,-54196.55837,52.620009,92.121191,0.0,0.0,1.0,0.0,0.0,0.0,0.0
58466,29070.66,29470.00,29010.36,29445.80,307.375737,1991.830678,29093.842,-53889.18263,55.830088,92.339769,0.0,0.0,1.0,0.0,0.0,0.0,0.0
58467,29445.80,29543.93,29366.86,29479.12,279.902558,1994.488683,29177.104,-53609.28008,54.413682,92.589984,0.0,0.0,1.0,0.0,0.0,0.0,0.0
58468,29484.39,29535.95,29296.95,29353.00,177.460767,1984.427829,29249.146,-53786.74084,54.313425,81.144377,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
data_features = data[['price_close', 'vader_pos', 'vader_neg']]

data_labels = data['price_close']

scaler_feature = MinMaxScaler(feature_range=(0, 1))
scaler_label = MinMaxScaler(feature_range=(0, 1))

feature = scaler_feature.fit_transform(data_features.to_numpy())
label = scaler_label.fit_transform(data_labels.to_numpy().reshape(-1, 1))

In [7]:
def split_data(f, l, delay):
    feature, label = [], []

    for i in range(len(f) - delay):
        feature.append(f[i: i + delay])
        label.append(l[i + delay-1])
        
    return np.array(feature), np.array(label).squeeze()

window_size = 1

feature, label = split_data(feature, label, window_size)


X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)

(13976, 1, 3) (3495, 1, 3)


In [8]:
model = \
    keras.Sequential([
        layers.Bidirectional(layers.GRU(50)),
        layers.Dropout(0.2),
        layers.Dense(1, activation='tanh'),
    ])

2022-11-04 16:59:17.991084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:59:17.999297: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:59:17.999509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 16:59:18.000167: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
EPOCHS = 500
BATCH_SIZE = 8

model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=3e-6), metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(X_train, y_train, 
                batch_size=BATCH_SIZE, 
                epochs=EPOCHS, 
                shuffle=False,
                )

2022-11-04 16:59:18.554761: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/500


2022-11-04 16:59:20.537982: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


1747/1747 [==============================] - 5s 2ms/step - loss: 0.1732 - root_mean_squared_error: 0.4162
Epoch 2/500
1747/1747 [==============================] - 3s 2ms/step - loss: 0.1202 - root_mean_squared_error: 0.3468
Epoch 3/500
1747/1747 [==============================] - 3s 2ms/step - loss: 0.0818 - root_mean_squared_error: 0.2860
Epoch 4/500
1747/1747 [==============================] - 3s 2ms/step - loss: 0.0552 - root_mean_squared_error: 0.2349
Epoch 5/500
1747/1747 [==============================] - 3s 2ms/step - loss: 0.0386 - root_mean_squared_error: 0.1966
Epoch 6/500
1747/1747 [==============================] - 3s 2ms/step - loss: 0.0297 - root_mean_squared_error: 0.1724
Epoch 7/500
1747/1747 [==============================] - 3s 2ms/step - loss: 0.0253 - root_mean_squared_error: 0.1589
Epoch 8/500
1747/1747 [==============================] - 3s 2ms/step - loss: 0.0228 - root_mean_squared_error: 0.1511
Epoch 9/500
1747/1747 [==============================] - 3s 2ms/step

In [10]:
prediction = np.array(model.predict(X_test))
pred_prices = scaler_label.inverse_transform(prediction)
true_prices = scaler_label.inverse_transform(y_test.reshape(-1,1))

mse = mean_squared_error(pred_prices, true_prices)
rmse = mean_squared_error(pred_prices, true_prices, squared=False)
mae = mean_absolute_error(pred_prices, true_prices)
mape = mean_absolute_percentage_error(pred_prices, true_prices)

print('mse:\t', mse)
print('rmse:\t', rmse)
print('mae:\t', mae)
print('mape:\t', mape)

mse:	 391500.8538195185
rmse:	 625.7002907299296
mae:	 439.89433214860514
mape:	 0.008765805262259783
